## Title 

Description

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "2_percent_v1"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

# Data distribution

In [4]:
labels = unpersist(filecontent_path)
for part in labels['partition'].unique():
    print(part)
    print(labels.query('partition == @part')['label'].value_counts())
    print()

train
Variable misuse    6985
Correct            6985
Name: label, dtype: int64

dev
Correct            731
Variable misuse    731
Name: label, dtype: int64

eval
Variable misuse    3741
Correct            3741
Name: label, dtype: int64



# Config

In [5]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    #use_edge_types=True,
    gpu=-1,
    epochs=2, 
    
    train_frac=0.8,
    random_seed=42, 
    
    # model parameters
    elem_emb_size=300,
    node_emb_size=300,                  # *** dimensionality of node embeddings
    h_dim=300,                           # *** should match to node dimensionality
    n_layers=5,
    dropout=0.1,
    activation="tanh"
)

In [6]:
save_config(config, "var_misuse_subgraph.yaml")

# Create Dataset

In [7]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [8]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [12]:
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective

class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SubgraphClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

## Launch Tensorboard

In [13]:
# %tensorboard --logdir './10_percent_v1'

## Start model training and Evaluation

In [14]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 999578


Epoch 1:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 0, Time: 731 s
{'Accuracy/test/VariableMisuseClf_': 0.5,
 'Accuracy/train/VariableMisuseClf_': 0.5,
 'Accuracy/train_avg/VariableMisuseClf': 0.4999289772727273,
 'Accuracy/val/VariableMisuseClf_': 0.5,
 'Loss/test/VariableMisuseClf_': 0.6931485321562169,
 'Loss/train/VariableMisuseClf_': 0.6931419372558594,
 'Loss/train_avg/VariableMisuseClf': 0.73564643480561,
 'Loss/val/VariableMisuseClf_': 0.6931487321853638,
 'grad_norm/train/_': 0.0012261094310606223}


Epoch 1: 100%|██████████| 110/110 [10:36<00:00,  5.78s/it]


Epoch: 1, Time: 915 s
{'Accuracy/test/VariableMisuseClf_': 0.5001324152542372,
 'Accuracy/train/VariableMisuseClf_': 0.5,
 'Accuracy/train_avg/VariableMisuseClf': 0.49957386363636364,
 'Accuracy/val/VariableMisuseClf_': 0.5,
 'Loss/test/VariableMisuseClf_': 0.6931503415107727,
 'Loss/train/VariableMisuseClf_': 0.6931900978088379,
 'Loss/train_avg/VariableMisuseClf': 0.6931503409689123,
 'Loss/val/VariableMisuseClf_': 0.6931506445010504,
 'grad_norm/train/_': 0.0014318958545823603}
Final eval: Loss/val/VariableMisuseClf_final: 0.6931506445010504, Accuracy/val/VariableMisuseClf_final: 0.5, Loss/test/VariableMisuseClf_final: 0.6931503415107727, Accuracy/test/VariableMisuseClf_final: 0.5001324152542372


(<__main__.Trainer at 0x2513871d8b0>,
 {'Loss/val/VariableMisuseClf_final': 0.6931506445010504,
  'Accuracy/val/VariableMisuseClf_final': 0.5,
  'Loss/test/VariableMisuseClf_final': 0.6931503415107727,
  'Accuracy/test/VariableMisuseClf_final': 0.5001324152542372})